In [4]:
# Get the current directory of the notebook file

import os

print(os.getcwd())


C:\Users\mdaminulisla.prodhan\All_My_Miscellenous\pubchempy


In [6]:
# Web Scrapping for getting the descriptors table from the mordred documentation

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch the webpage
response = requests.get('https://mordred-descriptor.github.io/documentation/master/descriptors.html')
soup = BeautifulSoup(response.text, 'html.parser')

# Find the tables on the page
tables = soup.find_all('table')

# This list will hold all of the data
data = []

# Iterate over each table
for table in tables:
    # Find all rows in this table
    rows = table.find_all('tr')
    
    # Iterate over each row
    for row in rows:
        # Find all columns in this row
        cols = row.find_all('td')
        
        # Get the text from each column and add it to the list
        cols = [col.text.strip() for col in cols]
        data.append(cols)

# Create a pandas DataFrame from the data
df = pd.DataFrame(data, columns=["#", "Module", "Name", "Constructor", "Dimension", "Description"])

# Write the DataFrame to an Excel file
df.to_excel("mordred_descriptors.xlsx", index=False)


In [9]:
# Generating files containing finally selected SMILES from U_SMILES.csv and SMILES and INCHIKEY from the 
# meged talbe on Approval_ID file

import pandas as pd

# Read the first CSV file "U_SMILES.csv"
df_smiles = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/U_SMILES.csv')

# Read the second CSV file "merged_on_Approval_ID.csv"
df_merged = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/merged_on_Approval_ID.csv')

# Merge the two dataframes based on the condition SMILES = SMILES1
merged_df = pd.merge(df_smiles, df_merged, how='inner', left_on='SMILES1', right_on='SMILES')

# Select the desired columns
result_df = merged_df[['SMILES1', 'SMILES', 'INCHIKEY']]

# Save the result to a new CSV file
result_df.to_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/pubchempy.csv', index=False)


In [10]:
# Deleting duplicate INCHIKEY from the "merged_on_Approval_ID.csv"

import pandas as pd

# Read the CSV file "merged_on_Approval_ID.csv"
df_merged = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/merged_on_Approval_ID.csv')

# Drop duplicate rows based on the "INCHIKEY" column
unique_df = df_merged.drop_duplicates(subset='INCHIKEY')

# Save the unique dataframe to a new CSV file
unique_df.to_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/unique_inchikeys.csv', index=False)


In [11]:
# Checking the effectiveness of deleting duplicate INCHIKEY by agian deleting the duplicate SMILES values 
import pandas as pd

# Read the CSV file "unique_inchikeys.csv"
df_unique_inchikeys = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/unique_inchikeys.csv')

# Drop duplicate rows based on the "SMILES" column
unique_smiles_df = df_unique_inchikeys.drop_duplicates(subset='SMILES')

# Save the unique SMILES dataframe to a new CSV file
unique_smiles_df.to_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/unique_smiles.csv', index=False)



In [12]:
# Final csv containing the identical SMILES and the corresponding INCHIKEY

import pandas as pd

# Read the CSV file "unique_inchikeys.csv"
df_unique_inchikeys = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/unique_inchikeys.csv')

# Read the CSV file "U_SMILES.csv"
df_usmiles = pd.read_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/U_SMILES.csv')

# Merge the dataframes based on the condition SMILES = SMILES1
merged_df = pd.merge(df_unique_inchikeys, df_usmiles, how='inner', left_on='SMILES', right_on='SMILES1')

# Select the desired columns
result_df = merged_df[['SMILES1', 'SMILES', 'INCHIKEY']]

# Save the result to a new CSV file
result_df.to_csv('C:/Users/mdaminulisla.prodhan/All_My_Miscellenous/pubchempy/result_final.csv', index=False)


In [15]:
# Convert the INCHIKEY to inchi string using pubchempy package 
import pandas as pd
import requests

# Read the input CSV file
df = pd.read_csv(r'C:\Users\mdaminulisla.prodhan\All_My_Miscellenous\pubchempy\result_final.csv')

# Create a new column for InChI
df['InChI'] = ''

# Iterate over the rows and retrieve InChI from PubChem using InChIKey
for index, row in df.iterrows():
    inchi_key = row['INCHIKEY']
    try:
        response = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{inchi_key}/property/InChI/txt")
        if response.status_code == 200:
            inchi = response.text.strip()
            df.at[index, 'InChI'] = inchi
        else:
            df.at[index, 'InChI'] = 'Error'
    except:
        df.at[index, 'InChI'] = 'Error'

# Save the updated DataFrame to a new CSV file
df.to_csv(r'C:\Users\mdaminulisla.prodhan\All_My_Miscellenous\pubchempy\result_final_with_inchi.csv', index=False)
